In [1]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from nltk.tag import pos_tag
from nltk.stem.snowball import SnowballStemmer
import re
from bs4 import BeautifulSoup
import requests

outfile = open('./news_document.txt', 'w', encoding='UTF8')
res = requests.get('https://www.nytimes.com/2018/12/03/upshot/why-hospitals-should-let-you-sleep.html?rref=collection%2Fsectioncollection%2Fscience&action=click&contentCollection=science&region=stream&module=stream_unit&version=latest&contentPlacement=6&pgtype=sectionfront')
soup = BeautifulSoup(res.content, "html.parser")

title_data = soup.find('h1')
outfile.write(title_data.get_text())

paragraph_data = soup.find_all('p')
for i in range(len(paragraph_data)):
    outfile.write(paragraph_data[i].get_text()+'\n')
outfile.close()

In [2]:
stemmer = SnowballStemmer("english")

filename = './news_document.txt'
s = ""
s1 = ''
sentence = []

with open(filename, 'r', encoding='UTF8') as f:
    for sen in f:
        s += sen.lower()
sentence = s.split('\n')

for i in range(len(sentence)):
    res = re.sub('[.]', '.\n', sentence[i])
    res = re.sub('[?]', '?\n', res)
    res += '\n'
    s1 += res
sentence_list = s.split('\n')

In [3]:
token_sentence = []
tagged_list = []
test = []
retokenize = RegexpTokenizer("[\w]+")
for i in sentence_list:
    token_sentence.append(retokenize.tokenize(i))
    tagged_list.append(pos_tag(word_tokenize((i))))

NN_list = []
NN_final = []

for i in range(len(tagged_list)):
    seg = []
    for idx, j in enumerate(tagged_list[i]):
        if tagged_list[i][idx][1] == 'NN' or tagged_list[i][idx][1] == 'NNP' or tagged_list[i][idx][1] == 'NNS':
            NN_list.append(stemmer.stem(tagged_list[i][idx][0]))
            seg.append((stemmer.stem(tagged_list[i][idx][0]), idx))
    NN_final.append(seg)

final_sentence = []
for i in range(len(tagged_list)):
    ex = []
    for j in range(len(tagged_list[i])):
        ex.append(tagged_list[i][j][0])
    final_sentence.append(ex)

for i in range(len(NN_final)):
    for j in range(len(NN_final[i])):
        final_sentence[i][NN_final[i][j][1]] = NN_final[i][j][0]
N = ' '.join(NN_list)
N = re.sub('[^a-zA-Z]', ' ', N)
N_N = N.strip().split(' ')
while '' in N_N:
    N_N.remove('')

counts = Counter(N_N)
max_NN = []
for i in range(5):
    max_NN.append(counts.most_common(n=20)[i][0])
print(max_NN)

['hospit', 'patient', 'disrupt', 'univers', 'medic']


In [4]:
count_list = []

for i in range(len(final_sentence)):
    count = 0
    for j in range(len(final_sentence[i])):
        if final_sentence[i][j] in max_NN:
            count += 1
    count_list.append(count)

number = max(count_list)
sentence_num = count_list.index(number)
print(sentence_list[sentence_num])

it’s more than annoying — such disruptions can harm patients. short sleep durations are associated with reduced immune function, delirium, hypertension and mood disorders. hospital conditions, including sleep disruptions, may contribute to “posthospital syndrome” — the period of vulnerability to a host of health problems after hospitalization that are not related to the reason for that hospitalization.
